# Training of CNN with 1x1 filter

Zašto korištenje 1x1 filtera?
Zato da algoritam pronađe vezu između true ishoda i dubine ulaza.

Planiram koristiti:
- INICIJALIZACIJA??
- aktivacijska funkcija: Relu
- normalizacija: Batch Normalization - zapravo, s obzirom da je Batch norm. koristan prvenstveno kod dubokih mreža, pokušati ću prvo bez toga, da vidim kako funkcionira.
- regularizacija: Dropout
- OPTIMIZER? ADAM?

S obzirom da ima malo podataka, razmisli može li se nekako povećati broj podataka data augmentation tehnikama.

In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
import csv

from sklearn.metrics import accuracy_score

# Common imports
import numpy as np
import os

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


%matplotlib inline

In [2]:
print ('DATA PREPARATION')
vege_csv = csv.reader(open('vegetacija.csv', newline=''), delimiter=' ', quotechar='|')

vege_array = []


for row in vege_csv:
	red_array = []
	items = row[0].split(';')
	for i in items:
		red_array.append(i)

	vege_array.append(red_array)


klase = []
values = []

vege_array_fix = vege_array[:-4]

print ('length of the fixed array is ', len (vege_array_fix))

for r in vege_array_fix:
    value_row = []
    i = 0
    while i < len(r):
        if i == 0:
            if r[i] == 'X':
                klase.append(0)
            if r[i] == 'V':
                klase.append(1)
            if r[i] == 'N':
                klase.append(2)
        else:
            value_row.append(float(r[i])/1000)
        i = i + 1
    values.append(value_row)

print ('test : ', values[10000])

DATA PREPARATION
length of the fixed array is  199400
test :  [0.339, 0.201, 0.238, 0.278, 0.186, 0.298, 0.298, 0.253]


In [3]:
X_train, X_test, y_train, y_test = train_test_split(values, klase, test_size=0.20)

In [4]:
reset_graph()

In [5]:
height = 1
width = 1
channels = 8
# n_inputs = height * width

conv1_fmaps = 16
conv1_ksize = 1
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 32
conv2_ksize = 1
conv2_stride = 1
conv2_pad = "SAME"

conv3_fmaps = 64
conv3_ksize = 1
conv3_stride = 1
conv3_pad = "SAME"

conv4_fmaps = 128
conv4_ksize = 1
conv4_stride = 1
conv4_pad = "SAME"

pool3_fmaps = conv4_fmaps

n_fc1 = 128
n_outputs = 3

# n_inputs = len(X_train)

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, channels], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.elu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.elu, name="conv2")
conv3 = tf.layers.conv2d(conv2, filters=conv3_fmaps, kernel_size=conv3_ksize,
                         strides=conv3_stride, padding=conv3_pad,
                         activation=tf.nn.elu, name="conv3") 
conv4 = tf.layers.conv2d(conv3, filters=conv4_fmaps, kernel_size=conv4_ksize,
                         strides=conv4_stride, padding=conv4_pad,
                         activation=tf.nn.elu, name="conv4") 


with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv4, ksize=[1, 1, 1, 1], strides=[1, 1, 1, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 1 * 1])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.elu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [6]:
print (len(X_train))

159520


In [7]:
X_valid = X_train[:63808]
X_train = X_train[63808:]

y_valid = y_train[:63808]
y_train = y_train[63808:]

In [8]:
train_array_length = len(X_train)
print (train_array_length)

95712


In [9]:
#moja egzekucijska funkcija

# n_epochs = 100
# batch_size = 20

# with tf.Session() as sess:
#     init.run()
#     for epoch in range(n_epochs):
#         for iteration in range(int(len(X_train)/batch_size)):
#             X_batch, y_batch = X_train.train.next_batch(batch_size)
#             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#         loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid, y: y_valid})
#         if loss_val < best_loss:
#             save_path = saver.save(sess, "./vege_model.ckpt")
#             best_loss = loss_val
#             checks_without_progress = 0
#         else:
#             checks_without_progress += 1
#             if checks_without_progress > max_checks_without_progress:
#                 print("Early stopping!")
#                 break
#         print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
#             epoch, loss_val, best_loss, acc_val * 100))

# with tf.Session() as sess:
#     saver.restore(sess, "./vege_model.ckpt")
#     acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
#     print("Final test accuracy: {:.2f}%".format(acc_test * 100))

In [10]:

# n_epochs = 10
# batch_size = 20

# max_checks_without_progress = 20
# checks_without_progress = 0
# best_loss = np.infty

# with tf.Session() as sess:
#     init.run()

#     for epoch in range(n_epochs):
#         rnd_idx = np.random.permutation(train_array_length)
#         for rnd_indices in np.array_split(rnd_idx, train_array_length // batch_size):
#             print (rnd_indices)

In [11]:

n_epochs = 1000
batch_size = 50

max_checks_without_progress = 40
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch = []
            y_batch = []
            for i in rnd_indices:
                X_batch.append(X_train[i]) 
                y_batch.append(y_train[i])
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid, y: y_valid})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./vege_model.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./vege_model.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.581432	Best loss: 0.581432	Accuracy: 81.84%
1	Validation loss: 0.582749	Best loss: 0.581432	Accuracy: 81.84%
2	Validation loss: 0.581815	Best loss: 0.581432	Accuracy: 81.84%
3	Validation loss: 0.573416	Best loss: 0.573416	Accuracy: 81.84%
4	Validation loss: 0.564402	Best loss: 0.564402	Accuracy: 81.84%
5	Validation loss: 0.564182	Best loss: 0.564182	Accuracy: 81.84%
6	Validation loss: 0.553595	Best loss: 0.553595	Accuracy: 81.87%
7	Validation loss: 0.577382	Best loss: 0.553595	Accuracy: 81.83%
8	Validation loss: 0.549103	Best loss: 0.549103	Accuracy: 81.88%
9	Validation loss: 0.544078	Best loss: 0.544078	Accuracy: 81.85%
10	Validation loss: 0.556329	Best loss: 0.544078	Accuracy: 81.84%
11	Validation loss: 0.538756	Best loss: 0.538756	Accuracy: 81.84%
12	Validation loss: 0.538914	Best loss: 0.538756	Accuracy: 81.88%
13	Validation loss: 0.541913	Best loss: 0.538756	Accuracy: 81.90%
14	Validation loss: 0.543655	Best loss: 0.538756	Accuracy: 81.88%
15	Validation loss: 

124	Validation loss: 0.455058	Best loss: 0.437956	Accuracy: 82.85%
125	Validation loss: 0.445651	Best loss: 0.437956	Accuracy: 83.04%
126	Validation loss: 0.448327	Best loss: 0.437956	Accuracy: 82.97%
127	Validation loss: 0.428388	Best loss: 0.428388	Accuracy: 83.42%
128	Validation loss: 0.450688	Best loss: 0.428388	Accuracy: 83.16%
129	Validation loss: 0.441413	Best loss: 0.428388	Accuracy: 83.07%
130	Validation loss: 0.454495	Best loss: 0.428388	Accuracy: 82.74%
131	Validation loss: 0.429200	Best loss: 0.428388	Accuracy: 83.41%
132	Validation loss: 0.442468	Best loss: 0.428388	Accuracy: 83.28%
133	Validation loss: 0.430382	Best loss: 0.428388	Accuracy: 83.45%
134	Validation loss: 0.429299	Best loss: 0.428388	Accuracy: 83.57%
135	Validation loss: 0.455229	Best loss: 0.428388	Accuracy: 82.30%
136	Validation loss: 0.425071	Best loss: 0.425071	Accuracy: 83.62%
137	Validation loss: 0.441551	Best loss: 0.425071	Accuracy: 83.23%
138	Validation loss: 0.431000	Best loss: 0.425071	Accuracy: 83

247	Validation loss: 0.401787	Best loss: 0.383936	Accuracy: 84.07%
248	Validation loss: 0.380828	Best loss: 0.380828	Accuracy: 84.86%
249	Validation loss: 0.378217	Best loss: 0.378217	Accuracy: 84.82%
250	Validation loss: 0.392758	Best loss: 0.378217	Accuracy: 84.48%
251	Validation loss: 0.380114	Best loss: 0.378217	Accuracy: 85.04%
252	Validation loss: 0.401302	Best loss: 0.378217	Accuracy: 84.34%
253	Validation loss: 0.381240	Best loss: 0.378217	Accuracy: 84.66%
254	Validation loss: 0.380534	Best loss: 0.378217	Accuracy: 84.64%
255	Validation loss: 0.414075	Best loss: 0.378217	Accuracy: 83.39%
256	Validation loss: 0.393897	Best loss: 0.378217	Accuracy: 84.44%
257	Validation loss: 0.388034	Best loss: 0.378217	Accuracy: 84.51%
258	Validation loss: 0.384191	Best loss: 0.378217	Accuracy: 85.03%
259	Validation loss: 0.386766	Best loss: 0.378217	Accuracy: 84.57%
260	Validation loss: 0.403729	Best loss: 0.378217	Accuracy: 83.84%
261	Validation loss: 0.412970	Best loss: 0.378217	Accuracy: 83

370	Validation loss: 0.409665	Best loss: 0.335007	Accuracy: 83.81%
371	Validation loss: 0.348025	Best loss: 0.335007	Accuracy: 85.97%
372	Validation loss: 0.342903	Best loss: 0.335007	Accuracy: 86.34%
373	Validation loss: 0.356202	Best loss: 0.335007	Accuracy: 85.97%
374	Validation loss: 0.351041	Best loss: 0.335007	Accuracy: 86.07%
375	Validation loss: 0.360656	Best loss: 0.335007	Accuracy: 85.50%
376	Validation loss: 0.334544	Best loss: 0.334544	Accuracy: 86.63%
377	Validation loss: 0.360370	Best loss: 0.334544	Accuracy: 85.71%
378	Validation loss: 0.374820	Best loss: 0.334544	Accuracy: 85.46%
379	Validation loss: 0.339268	Best loss: 0.334544	Accuracy: 86.38%
380	Validation loss: 0.344032	Best loss: 0.334544	Accuracy: 86.24%
381	Validation loss: 0.342935	Best loss: 0.334544	Accuracy: 86.28%
382	Validation loss: 0.350053	Best loss: 0.334544	Accuracy: 85.85%
383	Validation loss: 0.371491	Best loss: 0.334544	Accuracy: 85.55%
384	Validation loss: 0.372315	Best loss: 0.334544	Accuracy: 85